# **Chapter 04-b. Instance Segmentation (實例分割) & Semantic Segmentation (語意分割)**

* 上一節介紹的 `YOLO` 與 `RetinaNet` 都是 single-shot detection 的範例, 僅遍歷一次影像便產生結果
* 而另一種方法為, `使用第一個神經網路來建議要偵測物件的淺在位置, 再用第二個神經網路對這些位置進行微調與分類`, 這就是 `R-CNN` 的精神
* `Instance segmentation 就是抓出物件所在的所有像素, 以切割出他的輪廓`; 而 `semantic segmentation 就是將圖片中的每個像素都指派到一個類別, 而非以物件區分`
    * 例如：一張有三個人的圖片, `instance segmentation 會抓出三個不同的物件`, 但 `semantic segmentation 會把他們都當成人物類別的一部份`

### **Region Proposal Network (RPN, 區域提議網路)**
* RPN 是 two-stage 檢測器的「第一階段」, `從 feature map 上生成候選區域（region proposals）`, 用來縮小後續精細分類與回歸的計算範圍。
* RPN 只關心兩個類別：`物件` 和 `背景`, 它的兩個 heads 會分別用於`預測 bounding box`與`分析是物件還是背景`
* 輸入是 Backbone 輸出的特徵圖，而輸出是少量高品質的候選框（通常 ~200–300 個），供第二階段分類與回歸。
<br /><br />
* 運作流程：
    * Anchor 生成: 在 feature map 每個位置放置多種尺寸與比例的 anchor。(與 RetinaNet 的設定架構類似)
    * 二分類（物體 / 背景）: 小型分類器對每個 anchor 判斷是否可能包含物體。
    * 位置回歸: 預測 anchor 與真實框的偏移量。
    * 篩選（NMS）: 用 Non-Maximum Suppression 過濾重疊候選框，只保留最佳 ~200 個做為下一階段的 ROI。
<br /><br />
* 優缺點:
    * ✅ 提供高品質候選框，減少後續計算量
    * ✅ 通過共享 backbone，生成 proposals 幾乎不增加額外成本
    * ❌ 屬於 two-stage 架構，速度不如 one-stage（如 RetinaNet）



---
## **R-CNN <small>(Instance Segmentation)</small>**
* 影像會通過骨幹兩次, 第一次抓出 ROI 並裁減, 第二次對這些 ROI 的內容進行分類
* 但是, 如果直接裁減原圖在實務上太慢了, 因為 RPN 可以提議出 50 ~ 2000 個 ROI; 所以更好的做法是 `直接裁減 feature map`, 再對其執行 prediction head

<img src='Images/R-CNN.webp' width=1000 />


## **Mask R-CNN <small>(Instance Segmentation)</small>**
* 它除了能夠預測出 bounding box 外, 還可以預測出 `輪廓 & 物體在哪些像素點上（也就是遮罩 Mask）`
* 與 `YOLO、RetinaNet、R-CNN 只有兩個 prediction heads (classification head & bounding box regression head)` 不同, `Mask R-CNN 有第三個 Mask head`, 用來`對物件的單一像素進行分類 (抓出物件所在的像素, 這也就是所謂的實例分割 instance segmentation)`


#### Transposed Convolution / Deconvolution (轉置卷積 / 反卷積)
* 轉置卷積會執行 up-sampling 運算, 它與普通卷積使用相同的卷積矩陣, 只是現在`input 圖像的每一個 pixel 會被乘上 N × N filter 後加到 output 的 N × N 個 pixels`
* 轉置卷積被廣泛運用於生成影像的網路, 例如 autoencoder, GAN ...等, 但它的重疊部分容易產生 `棋盤假影(Checkerboard Artifacts)` (原因如下圖所示)

<img src='Images/Transposed_Conv_Checkboard_Artifacts.png' width=700 />

Transposed convolution 與 較推薦的 `up-convolution (最簡單的 nearest neighbor upsampling + conv 讓他更平滑, FPN 用到的那個方法)` 之比較:

<img src='Images/Transposed_Conv_vs_Up_Conv.png' width=700>

---
## **U-Net <small>(Semantic Segmentation)</small>**
* 架構上由 `對稱的 down-sampling encoder + up-sampling decoder` 組成
* Decoder side 負責抓出 mask 所的位置, 而它`與 encoder side 之間也有許多的 skip connection`, 他們以特定解析度來複製特徵, 將各種語意粒度(granularity)層級的資訊從 encoder 直接帶入 decoder

<img src='Images/U-Net.png' width=1000/>

### 延伸閱讀: 當前的深入研究方向
* EfficientDet
* DeepLabv3
* Panoptic FPN, Panoptic DeepLab